In [47]:
import pandas as pd
import numpy as np

import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load data
df = pd.read_csv('data/lead_scoring.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Prospect ID                                     9240 non-null   object 
 1   Lead Number                                     9240 non-null   int64  
 2   Lead Origin                                     9240 non-null   object 
 3   Lead Source                                     9204 non-null   object 
 4   Do Not Email                                    9240 non-null   object 
 5   Do Not Call                                     9240 non-null   object 
 6   Converted                                       9240 non-null   int64  
 7   TotalVisits                                     9103 non-null   float64
 8   Total Time Spent on Website                     9240 non-null   int64  
 9   Page Views Per Visit                     

In [48]:
# Contoh sebelum penggabungan
print("\nSebelum Penggabungan:")
print(df['Lead Origin'].value_counts())  # Menampilkan jumlah tiap value di kolom 'Lead Origin'

# Menggabungkan 'Quick Add Form' dan 'Lead Ads Form' menjadi 'Add Form'
df['Lead Origin'] = np.where(df['Lead Origin'].str.contains("Add Form", case=False), "Add Form", df['Lead Origin'])

# Contoh setelah penggabungan
print("\nSetelah Penggabungan:")
print(df['Lead Origin'].value_counts())  # Menampilkan hasil setelah penggabungan


Sebelum Penggabungan:
Lead Origin
Landing Page Submission    4886
API                        3580
Lead Add Form               718
Lead Import                  55
Quick Add Form                1
Name: count, dtype: int64

Setelah Penggabungan:
Lead Origin
Landing Page Submission    4886
API                        3580
Add Form                    719
Lead Import                  55
Name: count, dtype: int64


In [49]:
# Mengisi nilai yang hilang di kolom 'Lead Source' dengan 'Not Specified'
df['Lead Source'] = df['Lead Source'].fillna('Not Specified')

# Tampilkan persentase distribusi 'Lead Source'
lead_source_percentage = df['Lead Source'].value_counts(normalize=True)
print("\nSebelum Penggantian:")
print(lead_source_percentage)

# Tentukan threshold 1% (0.01) untuk menentukan kategori yang akan diganti menjadi 'Others'
threshold = 0.01
sources_to_replace = lead_source_percentage[lead_source_percentage < threshold].index  # Mendapatkan kategori yang kurang dari 1%

# Ganti 'Lead Source' yang kurang dari 1% dengan 'Others'
df['Lead Source'] = df['Lead Source'].apply(lambda x: 'Others' if x in sources_to_replace else x)

# Tampilkan persentase distribusi 'Lead Source' setelah penggantian
print("\nSetelah Penggantian:")
print(df['Lead Source'].value_counts(normalize=True))


Sebelum Penggantian:
Lead Source
Google               0.310390
Direct Traffic       0.275216
Olark Chat           0.189935
Organic Search       0.124892
Reference            0.057792
Welingak Website     0.015368
Referral Sites       0.013528
Facebook             0.005952
Not Specified        0.003896
bing                 0.000649
google               0.000541
Click2call           0.000433
Live Chat            0.000216
Social Media         0.000216
Press_Release        0.000216
Pay per Click Ads    0.000108
blog                 0.000108
WeLearn              0.000108
welearnblog_Home     0.000108
youtubechannel       0.000108
testone              0.000108
NC_EDM               0.000108
Name: proportion, dtype: float64

Setelah Penggantian:
Lead Source
Google              0.310390
Direct Traffic      0.275216
Olark Chat          0.189935
Organic Search      0.124892
Reference           0.057792
Welingak Website    0.015368
Referral Sites      0.013528
Others              0.012879
Name: p

In [50]:
# Untuk 'TotalVisits' dan 'Page Views Per Visit', kita lakukan modifikasi langsung tanpa menggunakan inplace
df['TotalVisits'] = df['TotalVisits'].fillna(0)  # Isi nilai kosong dengan 0 tanpa inplace
print("\nDeskripsi 'TotalVisits' setelah pengisian nilai kosong:")
print(df['TotalVisits'].describe())

df['Page Views Per Visit'] = df['Page Views Per Visit'].fillna(0)  # Isi nilai kosong dengan 0 tanpa inplace
print("\nDeskripsi 'Page Views Per Visit' setelah pengisian nilai kosong:")
print(df['Page Views Per Visit'].describe())


Deskripsi 'TotalVisits' setelah pengisian nilai kosong:
count    9240.000000
mean        3.394156
std         4.836682
min         0.000000
25%         0.000000
50%         3.000000
75%         5.000000
max       251.000000
Name: TotalVisits, dtype: float64

Deskripsi 'Page Views Per Visit' setelah pengisian nilai kosong:
count    9240.000000
mean        2.327787
std         2.164258
min         0.000000
25%         0.000000
50%         2.000000
75%         3.000000
max        55.000000
Name: Page Views Per Visit, dtype: float64


In [51]:
# Pastikan nilai kosong (null) pada 'Last Activity' ditangani
df['Last Activity'] = df['Last Activity'].fillna('Unknown')  # Mengisi nilai NaN dengan 'Unknown'

# Tampilkan jumlah setiap aktivitas pada kolom 'Last Activity'
display(df['Last Activity'].value_counts())

# Definisikan kategori berdasarkan sudut pandang perusahaan (subjektif)
good_activities = ['Email Opened', 'Resubscribed to emails',  'Page Visited on Website', 'Form Submitted on Website', 'Email Link Clicked', 'Had a Phone Conversation', 'Approached upfront']
bad_activities = ['Email Bounced', 'Unreachable', 'Unsubscribed', 'Email Marked Spam']
neutral_activities = ['Email Received', 'SMS Sent', 'Olark Chat Conversation', 'Converted to Lead', 'View in browser link Clicked', 'Visited Booth in Tradeshow']

# Fungsi untuk mengkategorikan aktivitas menjadi Good, Bad, atau Neutral
def categorize_activity(activity):
    if activity in good_activities:
        return 'Good'
    elif activity in neutral_activities:
        return 'Neutral'
    elif activity in bad_activities:
        return 'Bad'
    else:
        return 'Bad'  # Jika aktivitas tidak dikenal atau 'Unknown', dianggap Bad

# Terapkan fungsi ke kolom 'Last Activity' untuk membuat kolom kategori
df['Last Activity Category'] = df['Last Activity'].apply(categorize_activity)

# Tampilkan hasil kategori 'Last Activity'
last_activity_category_counts = df['Last Activity Category'].value_counts()
display(last_activity_category_counts)

# Drop kolom 'Last Activity' karena tidak lagi diperlukan
df = df.drop(columns=['Last Activity'])


Last Activity
Email Opened                    3437
SMS Sent                        2745
Olark Chat Conversation          973
Page Visited on Website          640
Converted to Lead                428
Email Bounced                    326
Email Link Clicked               267
Form Submitted on Website        116
Unknown                          103
Unreachable                       93
Unsubscribed                      61
Had a Phone Conversation          30
Approached upfront                 9
View in browser link Clicked       6
Email Marked Spam                  2
Email Received                     2
Resubscribed to emails             1
Visited Booth in Tradeshow         1
Name: count, dtype: int64

Last Activity Category
Good       4500
Neutral    4155
Bad         585
Name: count, dtype: int64

In [52]:
# Tampilkan jumlah data pada kolom 'Country' sebelum diubah
print("\nBefore")
display(df['Country'].value_counts(dropna=False))  # Menggunakan dropna=False agar nilai NaN juga ditampilkan

# Ubah nilai 'Country' menjadi 'Indonesia' atau 'Outside Indonesia', termasuk yang Null/None
df['Country'] = np.where(df['Country'] == 'Indonesia', 'Indonesia', 'Outside Indonesia')

# Tampilkan jumlah data pada kolom 'Country' setelah diubah
print("\nAfter")
display(df['Country'].value_counts())


Before


Country
Indonesia               6492
NaN                     2461
United States             69
United Arab Emirates      53
Singapore                 24
Saudi Arabia              21
United Kingdom            15
Australia                 13
Qatar                     10
Hong Kong                  7
Bahrain                    7
Oman                       6
France                     6
unknown                    5
Kuwait                     4
South Africa               4
Nigeria                    4
Germany                    4
Canada                     4
Sweden                     3
Italy                      2
Belgium                    2
Ghana                      2
Philippines                2
Netherlands                2
China                      2
Asia/Pacific Region        2
Bangladesh                 2
Uganda                     2
Russia                     1
Sri Lanka                  1
Tanzania                   1
Kenya                      1
Malaysia                   1
Liberi


After


Country
Indonesia            6492
Outside Indonesia    2748
Name: count, dtype: int64

In [53]:
# Tampilkan jumlah data pada kolom 'Specialization' sebelum diubah
print("\nBefore")
display(df['Specialization'].value_counts(dropna=False))  # Tampilkan juga nilai NaN

# Ubah nilai 'Select' dan yang Null/None menjadi 'Not Specified'
df['Specialization'] = df['Specialization'].replace('Select', 'Not Specified').fillna('Not Specified')

# Tampilkan jumlah data pada kolom 'Specialization' setelah diubah
print("\nAfter")
display(df['Specialization'].value_counts())


Before


Specialization
Select                               1942
NaN                                  1438
Finance Management                    976
Human Resource Management             848
Marketing Management                  838
Operations Management                 503
Business Administration               403
IT Projects Management                366
Supply Chain Management               349
Banking, Investment And Insurance     338
Media and Advertising                 203
Travel and Tourism                    203
International Business                178
Healthcare Management                 159
Hospitality Management                114
E-COMMERCE                            112
Retail Management                     100
Rural and Agribusiness                 73
E-Business                             57
Services Excellence                    40
Name: count, dtype: int64


After


Specialization
Not Specified                        3380
Finance Management                    976
Human Resource Management             848
Marketing Management                  838
Operations Management                 503
Business Administration               403
IT Projects Management                366
Supply Chain Management               349
Banking, Investment And Insurance     338
Media and Advertising                 203
Travel and Tourism                    203
International Business                178
Healthcare Management                 159
Hospitality Management                114
E-COMMERCE                            112
Retail Management                     100
Rural and Agribusiness                 73
E-Business                             57
Services Excellence                    40
Name: count, dtype: int64